## Compare data outputs from two different observatory instances on the same input data
This is useful for regression testing before deploying to prod / staging

### Area Occupancy
Compare area occupancy counts

#### Retrieve data
- Adjust the time filter as necessary
- Get from prod and from local
- Connect to db with psql and run the following command (the client needs to run on your machine, because the output file path is relative to the client)
```psql
\copy (select * from areaoccupancy a join metadata m on m.id = a.metadata_id where a.occupancy_time > '2025-12-10' order by a.occupancy_time) to 'local-area.csv' with csv header
```

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Whenever you change the code in imported modules, just rerun this cell, autoreload makes restarting the kernel unnecessary

from common import read_area_csv

In [ ]:
MECKAUER_AREAS = [
    'parking-meckauer-west01',
    'parking-meckauer-west02',
    'parking-meckauer-west03',
    'parking-meckauer-center',
    'parking-meckauer-center02',
    'parking-meckauer-ost01',
    'parking-meckauer-ost02',
]

local_df = read_area_csv('local-area.csv', MECKAUER_AREAS)
prod_df = read_area_csv('prod-area.csv', MECKAUER_AREAS)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=local_df.index, 
        y=local_df['total_count'],
        mode='lines',
        name='local'
    ),
)
fig.add_trace(
    go.Scattergl(
        x=prod_df.index, 
        y=prod_df['total_count'],
        mode='lines',
        name='prod'
    ),
)


fig.update_layout(
    title='Count over Time',
    xaxis_title='time',
    yaxis_title='total_count',
)

fig.show()